In [79]:
import cv2
import glob
import random
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV


In [88]:
metascore = []

In [90]:
#emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]

In [91]:
clf = ExtraTreesClassifier(criterion='gini',
                           max_depth=6, 
                           max_features=98, 
                           min_samples_leaf=7,
                           min_samples_split=16, 
                           n_estimators=200
                          )

In [92]:
def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


In [93]:
def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


In [94]:
training_X, training_y, test_X, test_y = make_sets()

training_X = training_X.reshape(training_X.shape[0], 350*350)
test_X = test_X.reshape(test_X.shape[0], 350*350)

In [95]:
for i in range(0, 10):
    clf = clf.fit(training_X, training_y)
    predict = clf.predict(test_X)
    score = clf.score(test_X, test_y)
    metascore.append(score)
    
metascore = np.array(metascore)

print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(metascore.mean()*100) + '\033[0m')
print ('\033[1m Margin of Erro: ' + '\033[1;34m {} \n'.format(metascore.std()*100) + '\033[0m')

metascore = []

 Score:  97.55813953488371 

 Margin of Erro:  0.81395348837209 



In [97]:
#cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])
cm = confusion_matrix(test_y, predict, labels=[5,7])
    
print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

ValueError: At least one label specified must be in y_true

In [21]:
param_dist = {"max_depth" :  [20, 35, 50],
               "max_features" : sp_randint(15, 18),
               "min_samples_split" : sp_randint(12, 14),
               "min_samples_leaf" : sp_randint(5, 9),
               "n_estimators" : [500, 600, 700],
               "criterion" : ["gini", "entropy"]
              }

grid = RandomizedSearchCV(estimator=clf, param_distributions = param_dist, n_iter=100)
grid.fit(training_X, training_y)

y_true, y_pred = test_y, grid.predict(test_X)
bs = grid.best_score_
bp = grid.best_params_
cr = classification_report(y_true, y_pred)

print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')
print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))
print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

 Best Score:  0.5864951768488746

 Best Params:

 {'criterion': 'gini', 'max_depth': 35, 'max_features': 17, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 700}

 Classification Report:

              precision    recall  f1-score   support

          0       0.57      1.00      0.73       193
          1       0.00      0.00      0.00        26
          2       0.00      0.00      0.00        10
          3       0.00      0.00      0.00        35
          4       0.00      0.00      0.00        14
          5       0.88      0.54      0.67        41
          6       0.00      0.00      0.00        21
          7       1.00      0.51      0.68        45

avg / total       0.50      0.62      0.52       385



C:\Users\hermi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
